# Análise de Dados Aprendizado de Máquina - NBA

_descrever objetivo do trabalho e etc_

## Informações do Aluno:

* Nome: Gabriel Castelo Branco Rocha Alencar Pinto
* Matrícula: 2020006523


## Aprendizado supervisionado



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import modules.supervisioned as sup
import importlib

plt.style.use("seaborn")

plt.rcParams['figure.figsize']  = (16, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4


def despine(ax=None):
    if ax is None:
        ax = plt.gca()
        
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
%load_ext autoreload
%autoreload 2

/tmp/ipykernel_26269/936430673.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


In [2]:
df_treino = pd.read_csv("./data/nba_treino.csv")
df_treino.head()

df_treino['TARGET_5Yrs'] = df_treino["TARGET_5Yrs"].astype(bool)
df_treino

,GP,MIN,PTS,FGM,FGA,FG%,3PMade,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,False
1,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,False
2,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,False
3,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,True
4,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,80,15.8,4.3,1.6,3.6,43.3,0.0,0.2,14.3,1.2,1.5,79.2,0.4,0.8,1.2,2.5,0.6,0.2,0.8,False
1068,68,12.6,3.9,1.5,4.1,35.8,0.1,0.7,16.7,0.8,1.0,79.4,0.4,1.1,1.5,2.3,0.8,0.0,1.3,True
1069,43,12.1,5.4,2.2,3.9,55.0,0.0,0.0,0.0,1.0,1.6,64.3,1.5,2.3,3.8,0.3,0.3,0.4,0.9,False
1070,52,12.0,4.5,1.7,3.8,43.9,0.0,0.2,10.0,1.2,1.8,62.5,0.2,0.4,0.7,2.2,0.4,0.1,0.8,True


In [3]:
df_teste = pd.read_csv("./data/nba_teste.csv")
df_teste.head()

df_teste['TARGET_5Yrs'] = df_teste["TARGET_5Yrs"].astype(bool)

In [4]:
classifier2 = sup.KNNClassifier(df_treino, "TARGET_5Yrs", 2)
classifier2.train()

classifier10 = sup.KNNClassifier(df_treino, "TARGET_5Yrs", 10)
classifier10.train()

classifier25 = sup.KNNClassifier(df_treino, "TARGET_5Yrs", 25)
classifier25.train()

classifier50 = sup.KNNClassifier(df_treino, "TARGET_5Yrs", 50)
classifier50.train()



In [5]:
i = classifier2.predict(df_teste)
count = 0

for j in i:
    if j.label == j.estimated_label:
        count += 1
                
per = count/len(i)
print(f"Percentual de acerto: {(per * 100):.2f}%")

Percentual de acerto: 61.57%


In [6]:
classifs = [classifier2, classifier10, classifier25, classifier50]
a = []

for i in classifs:
    a.append(i.predict(df_teste))

In [7]:
for idx, i in enumerate(a):
    count = 0   
    
    for j in i:
        if j.label == j.estimated_label:
            count += 1
                    
    per = count/len(i)
    print(f"Percentual de acerto (K = {classifs[idx].K}): {(per * 100):.2f}%")

Percentual de acerto (K = 2): 61.57%
Percentual de acerto (K = 10): 66.04%
Percentual de acerto (K = 25): 65.67%
Percentual de acerto (K = 50): 68.66%
